In [11]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv
/kaggle/input/house-prices-advanced-regression-techniques/data_description.txt
/kaggle/input/house-prices-advanced-regression-techniques/train.csv
/kaggle/input/house-prices-advanced-regression-techniques/test.csv


In [13]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import itertools

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pylab import rcParams
import matplotlib

import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
import tensorflow as tf
from tensorflow import keras


from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

## Importing numerical data

In [14]:
train = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/train.csv')
test = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/test.csv')

df=pd.concat([train,test])
# df.set_inde
df.reset_index(drop = True, inplace = True)
df.drop('Id',axis = 1, inplace = True)
df.fillna(0,inplace=True)
df=df.select_dtypes(exclude=['object'])
df.info()
# df.shape,display(df)
# df.iloc[120/0:1500]
# train
# test

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2919 entries, 0 to 2918
Data columns (total 37 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   MSSubClass     2919 non-null   int64  
 1   LotFrontage    2919 non-null   float64
 2   LotArea        2919 non-null   int64  
 3   OverallQual    2919 non-null   int64  
 4   OverallCond    2919 non-null   int64  
 5   YearBuilt      2919 non-null   int64  
 6   YearRemodAdd   2919 non-null   int64  
 7   MasVnrArea     2919 non-null   float64
 8   BsmtFinSF1     2919 non-null   float64
 9   BsmtFinSF2     2919 non-null   float64
 10  BsmtUnfSF      2919 non-null   float64
 11  TotalBsmtSF    2919 non-null   float64
 12  1stFlrSF       2919 non-null   int64  
 13  2ndFlrSF       2919 non-null   int64  
 14  LowQualFinSF   2919 non-null   int64  
 15  GrLivArea      2919 non-null   int64  
 16  BsmtFullBath   2919 non-null   float64
 17  BsmtHalfBath   2919 non-null   float64
 18  FullBath

## Outliers

In [15]:
from sklearn.ensemble import IsolationForest

clf = IsolationForest(max_samples = 100, random_state = 42)
clf.fit(df)
y_noano = clf.predict(df)
y_noano = pd.DataFrame(y_noano, columns = ['Top'])
y_noano[y_noano['Top'] == 1].index.values

df = df.iloc[y_noano[y_noano['Top'] == 1].index.values]
df.reset_index(drop = True, inplace = True)
print("Number of Outliers:", y_noano[y_noano['Top'] == -1].shape[0])
print("Number of rows without outliers:", df.shape[0])

Number of Outliers: 289
Number of rows without outliers: 2630


In [16]:
FEATURES= [col for col in df.columns if col != "SalePrice"]

# dff = scaler.fit_transform(df.to_numpy())
# df = pd.DataFrame(dff, columns=df.columns)

X=df[FEATURES]
y=df['SalePrice']
# print(y.shape,X.shape)
# len(y.to_numpy())
# X_scaler = MinMaxScaler()
# X_arr = X_scaler.fit_transform(X.to_numpy())
# X_scaled = pd.DataFrame(X_arr, columns=X.columns)
# # X
# y_scaler = MinMaxScaler()
# y_arr = y_scaler.fit_transform(y.to_numpy().reshape(2630,1))
# y_scaled = pd.DataFrame(y_arr, columns=['SalePrice'])

# y.to_numpy().reshape(2630,1)
# y.columns
# y

x_train, x_test, y_train, y_test = train_test_split(X , y, test_size=0.33, random_state=42)



In [18]:
normalizer = tf.keras.layers.Normalization(axis=-1)
normalizer.adapt(np.array(X))
print(normalizer.mean.numpy())

[[5.5433468e+01 5.6998848e+01 9.6167148e+03 6.0673013e+00 5.5802283e+00
  1.9727299e+03 1.9844248e+03 9.1538437e+01 4.2843729e+02 4.4592785e+01
  5.5465894e+02 1.0276892e+03 1.1289817e+03 3.1053159e+02 2.6980999e+00
  1.4422120e+03 4.2053235e-01 5.5133071e-02 1.5368818e+00 3.6844116e-01
  2.8064640e+00 1.0273763e+00 6.2870722e+00 5.7224345e-01 1.9328741e+03
  1.7802285e+00 4.7348904e+02 9.0319054e+01 4.3875294e+01 1.9070721e+01
  2.7912548e+00 1.4902666e+01 8.6083645e-01 3.8798096e+01 6.1730061e+00
  2.0077806e+03]]


In [27]:

# seed = 7
# np.random.seed(seed)

model = tf.keras.Sequential([
    normalizer,
    Dense(32,  activation = 'exponential'),

    Dense(64, activation='relu'),
    Dense(1)
])

model.compile(loss='mean_absolute_error',
                optimizer=tf.keras.optimizers.Adam(0.001))

# # Model
# model = Sequential()
# model.add(Dense(200, input_dim=36, kernel_initializer='normal', activation='relu'))
# model.add(Dense(100, kernel_initializer='normal', activation='relu'))
# model.add(Dense(50, kernel_initializer='normal', activation='relu'))
# model.add(Dense(25, kernel_initializer='normal', activation='relu'))
# model.add(Dense(1, kernel_initializer='normal'))
# # Compile model
# model.compile(loss='mean_squared_error', optimizer=keras.optimizers.Adadelta())

# feature_cols = training_set[FEATURES]
# labels = training_set[LABEL].values

model.fit(x_train, y_train, epochs=500, batch_size=10)

Epoch 1/500
177/177 [==============================] - 1s 2ms/step - loss: 86086.8438
Epoch 2/500
177/177 [==============================] - 0s 2ms/step - loss: 86084.3672
Epoch 3/500
177/177 [==============================] - 0s 2ms/step - loss: 86069.9688
Epoch 4/500
177/177 [==============================] - 0s 2ms/step - loss: 86011.6484
Epoch 5/500
177/177 [==============================] - 0s 2ms/step - loss: 86173.3047
Epoch 6/500
177/177 [==============================] - 0s 2ms/step - loss: 85954.6484
Epoch 7/500
177/177 [==============================] - 0s 2ms/step - loss: 85487.4922
Epoch 8/500
177/177 [==============================] - 0s 2ms/step - loss: 85129.8828
Epoch 9/500
177/177 [==============================] - 0s 2ms/step - loss: 85123.1797
Epoch 10/500
177/177 [==============================] - 0s 2ms/step - loss: 84749.5859
Epoch 11/500
177/177 [==============================] - 0s 2ms/step - loss: 84219.3594
Epoch 12/500
177/177 [==============================

KeyboardInterrupt: 

In [22]:
model.evaluate(x_test, y_test)


28/28 [==============================] - 0s 2ms/step - loss: 93081.5078


93081.5078125

In [23]:
tes=(X.iloc[0:100,:]).to_numpy()
pre=model.predict_on_batch(X_scaler.transform(tes))
# pre

# pd.concat([pd.Series(y_scaler.inverse_transform(pre)) ,y[0:100] ],axis=1)
data={'pred':y_scaler.inverse_transform(pre).reshape(1,-1)[0] , 'org':y[0:100].to_numpy()}
pd.DataFrame( data=data)
df.loc[0,['SalePrice']]
# y.iloc[0:100]
# pd.Series(y_scaler.inverse_transform(pre)) 
# y[0:100]
# y_scaler.inverse_transform(pre).reshape(1,-1)[0]
# y[0:100].to_numpy()

NameError: name 'X_scaler' is not defined